In [1]:
import numpy as np
import pandas as pd
import os
import random


CG

In [13]:
l = []
for group in range(1,11):
    print(group)
    cg = pd.read_csv(f"Group_{group:02d}/GROUP_{group:02d}_CG.csv")
    print(cg.iloc[:, 3:-1].sum().sum())
    l.append(cg.iloc[:, 3:-1].sum().sum())
print(f"average is {sum(l)/len(l)}")


1
32
2
26
3
26
4
19
5
23
6
23
7
26
8
46
9
22
10
46
average is 28.9


In [10]:
cg.iloc[:, 3:-1].sum().sum()

46

In [3]:
# Map only the cga with the longest duration

for group in range(1,11):
    print(group)
    cg = pd.read_csv(f"Group_{group:02d}/Group_{group:02d}_merge.csv")
    # print(len(com))
    segments = pd.read_csv(f"Group_{group:02d}/Group_{group:02d}_Oracle.csv")


    mapped_rows = []

    # Step 3: Get the columns to map from the "cps" DataFrame (excluding "Start," "End," and "Utterance")
    # columns_to_map = cg.columns.difference(["Begin Time - ss.msec", "End Time - ss.msec"])
    columns_to_map = ["CGA"]

    # Step 4: Iterate through each row in the "segments" DataFrame
    for _, segment_row in segments.iterrows():
        segment_start = segment_row["Start"]
        segment_end = segment_row["End"]
        
        # Step 5: Find overlapping rows in the "cps" DataFrame
        overlapping_rows = cg[(cg["End Time - ss.msec"] >= segment_start) & (cg["Begin Time - ss.msec"] <= segment_end)]
        overlapping_rows = overlapping_rows.sort_values(by="Duration - ss.msec")
        if not overlapping_rows.empty:
            # Step 6: Initialize a dictionary to store mapped data
            mapped_data = {
                "Utterance": segment_row["Utterance"],
                "Start": segment_start,
                "End": segment_end
            }
            
            # Step 7: Map the values from the "cg" DataFrame to the "segments" row
            for col in columns_to_map:
                mapped_data[col] = overlapping_rows.iloc[0][col]
            
            # Step 8: Append the mapped data to the list of mapped rows
            mapped_rows.append(mapped_data)
        else:
            # If there are no overlapping rows, create a mapping with empty values
            mapped_data = {
                "Utterance": segment_row["Utterance"],
                "Start": segment_start,
                "End": segment_end
            }
            for col in columns_to_map:
                mapped_data[col] = None
            mapped_rows.append(mapped_data)

    # Step 9: Create a new DataFrame with the mapped rows
    mapped_df = pd.DataFrame(mapped_rows)
    mapped_df[["Observation", "Inference", "Statement", "Accept", "Doubt", "Question", "Recommendation", "Absence"]] = 0
    for i in range(len(mapped_df)):
        if mapped_df.iloc[i]["CGA"]:
            if "OBSERVATION" in mapped_df.iloc[i]["CGA"]:
                mapped_df.at[i, "Observation"] = 1
            elif "STATEMENT" in mapped_df.iloc[i]["CGA"]:
                mapped_df.at[i, "Statement"] = 1
            elif "INFERENCE" in mapped_df.iloc[i]["CGA"]:
                mapped_df.at[i, "Inference"] = 1
            elif "ACCEPT" in mapped_df.iloc[i]["CGA"]:
                mapped_df.at[i, "Accept"] = 1
            elif "DOUBT" in mapped_df.iloc[i]["CGA"]:
                mapped_df.at[i, "Doubt"] = 1
            elif "QUESTION" in mapped_df.iloc[i]["CGA"]:
                mapped_df.at[i, "Question"] = 1
            elif "RECOMMENDATION" in mapped_df.iloc[i]["CGA"]:
                mapped_df.at[i, "Recommendation"] = 1
        else:
            mapped_df.at[i, "Absence"] = 1

    mapped_df = mapped_df.drop("CGA", axis=1)



    mapped_df.sort_values(by="Utterance").to_csv(f"Group_{group:02d}/GROUP_{group:02d}_CG.csv", index=False)

1
2
3
4
5
6
7
8
9
10


In [ ]:
count = 0
count_cg = 0
for group in range(1,11):
    
    com = pd.read_csv(f"Group_{group:02d}/GROUP_{group:02d}_merge.csv")
    print(len(com))
    count_cg += len(com)
    segments = pd.read_csv(f"Group_{group:02d}/Group_{group:02d}_Oracle.csv")
    seg_com = segments.copy().iloc[:,:4]
    seg_com[["Observation", "Inference", "Statement", "Accept", "Doubt", "Question", "Recommendation"]] = 0
    
    ptr = 0
    for i in range(com.shape[0]):
        start, end = float(com.iloc[i, 0]), float(com.iloc[i, 1])
        while True:
            if start > float(seg_com.iloc[ptr, 2]) or (float(seg_com.iloc[ptr, 2]) - start) < (end - seg_com.iloc[min(ptr+1, seg_com.shape[0]-1), 1]):
                ptr += 1
            else:
                if end < seg_com.iloc[ptr, 1]:
                    utterance_id = ptr + 1000
                else:
                    utterance_id = ptr
                break
        com.at[i, "Utterance_id"] = int(utterance_id)
    
    for i in range(com.shape[0]):
        idx = int(com.iloc[i]["Utterance_id"])
        if "OBSERVATION" in com.iloc[i]["CGA"]:
            seg_com.at[idx, "Observation"] = 1
        elif "STATEMENT" in com.iloc[i]["CGA"]:
            seg_com.at[idx, "Statement"] = 1
        elif "INFERENCE" in com.iloc[i]["CGA"]:
            seg_com.at[idx, "Inference"] = 1
        elif "ACCEPT" in com.iloc[i]["CGA"]:
            seg_com.at[idx, "Accept"] = 1
        elif "DOUBT" in com.iloc[i]["CGA"]:
            seg_com.at[idx, "Doubt"] = 1
        elif "QUESTION" in com.iloc[i]["CGA"]:
            seg_com.at[idx, "Question"] = 1
        elif "RECOMMENDATION" in com.iloc[i]["CGA"]:
            seg_com.at[idx, "Recommendation"] = 1
    seg_com.to_csv(f"Group_{group:02d}/Group_{group:02d}_CG.csv", index=False)
    print(seg_com.iloc[:, 4:].sum().astype(int))
    print()
    count += seg_com.iloc[:, 4:].sum().astype(int)


CPS

In [63]:
for group in range(1,11):
    print(group)
    cps = pd.read_csv(f"Group_{group:02d}/GROUP_{group:02d}_CPS.csv")
    # print(len(com))
    segments = pd.read_csv(f"Group_{group:02d}/Group_{group:02d}_Oracle.csv")


    mapped_rows = []

    # Step 3: Get the columns to map from the "cps" DataFrame (excluding "Start," "End," and "Utterance")
    columns_to_map = cps.columns.difference(["Start", "End", "Utterance"])

    # Step 4: Iterate through each row in the "segments" DataFrame
    for _, segment_row in segments.iterrows():
        segment_start = segment_row["Start"]
        segment_end = segment_row["End"]
        
        # Step 5: Find overlapping rows in the "cps" DataFrame
        overlapping_rows = cps[(cps["End"] >= segment_start) & (cps["Start"] <= segment_end)]
        
        if not overlapping_rows.empty:
            # Step 6: Initialize a dictionary to store mapped data
            mapped_data = {
                "Utterance": segment_row["Utterance"],
                "Start": segment_start,
                "End": segment_end
            }
            
            # Step 7: Map the values from the "cps" DataFrame to the "segments" row
            for col in columns_to_map:
                mapped_data[col] = overlapping_rows[col].values[0]
            
            # Step 8: Append the mapped data to the list of mapped rows
            mapped_rows.append(mapped_data)
        else:
            # If there are no overlapping rows, create a mapping with empty values
            mapped_data = {
                "Utterance": segment_row["Utterance"],
                "Start": segment_start,
                "End": segment_end
            }
            for col in columns_to_map:
                mapped_data[col] = None
            mapped_rows.append(mapped_data)

    # Step 9: Create a new DataFrame with the mapped rows
    mapped_df = pd.DataFrame(mapped_rows)

    mapped_df.sort_values(by="Utterance").to_csv(f"Group_{group:02d}/GROUP_{group:02d}_CPS_Oracle.csv", index=False, columns=cps.columns)

1
2
3
4
5
6
7
8
9
10


GAMR

In [73]:
for group in range(1,11):
    print(group)
    gamr = pd.read_csv(f"GROUP_{group:02d}_GAMR_RB.csv")
    # print(len(com))
    segments = pd.read_csv(f"Group_{group:02d}/Group_{group:02d}_Oracle.csv")
    # seg_com = segments.copy().iloc[:,:4]

    

    mapped_rows = []
    
    # Step 3: Initialize a set to keep track of mapped "oracle" row indices
    mapped_indices = set()

    # Step 3: Iterate through each row in the "gamr" DataFrame
    for _, gamr_row in gamr.iterrows():
        gamr_begin = gamr_row["Begin Time - ss.msec"]
        gamr_end = gamr_row["End Time - ss.msec"]
        
        # Step 4: Find overlapping rows in the "oracle" DataFrame
        overlapping_rows = segments[(segments["End"] >= gamr_begin) & (segments["Start"] <= gamr_end)]
        
        if not overlapping_rows.empty:
            # Step 5: Map to the row with the maximum overlap
            max_overlap_row = overlapping_rows.loc[(overlapping_rows["End"] - overlapping_rows["Start"]).idxmax()]
            mapped_row = {
                "Utterance": max_overlap_row["Utterance"],
                "Start": max_overlap_row["Start"],
                "End": max_overlap_row["End"],
                "Participant_1": gamr_row["Participant_1"],
                "Participant_2": gamr_row["Participant_2"],
                "Participant_3": gamr_row["Participant_3"]
            }
            mapped_rows.append(mapped_row)

            mapped_indices.add(max_overlap_row.index[0])
    # Step 8: Iterate through "oracle" rows that were not mapped and add them to the final output
    for idx, oracle_row in segments.iterrows():
        if idx not in mapped_indices:
            mapped_row = {
                "Utterance": oracle_row["Utterance"],
                "Start": oracle_row["Start"],
                "End": oracle_row["End"],
                "Participant_1": "",
                "Participant_2": "",
                "Participant_3": ""
            }
            mapped_rows.append(mapped_row)

    # Create a new DataFrame with the mapped rows
    mapped_df = pd.DataFrame(mapped_rows)

    mapped_df = mapped_df.sort_values(by="Utterance")

    # Save the result to a new CSV file
    mapped_df.to_csv(f"Group_{group:02d}/GROUP_{group:02d}_GAMR_Oracle.csv", index=False)


1
2
3
4
5
6
7
8
9
10


In [21]:
non_overlapping = []
n_gamr = 0
n_segments = 0
for group in range(1,11):
    # print(group)
    gamr = pd.read_csv(f"GAMR/GROUP_{group:02d}_GAMR_RB.csv")
    n_gamr += len(gamr)
    segments = pd.read_csv(f"Group_{group:02d}/Group_{group:02d}_Oracle.csv")
    n_segments += len(segments)

    mapped_rows = []
    
    # Step 3: Initialize a set to keep track of mapped "oracle" row indices
    mapped_indices = set()
    # Step 3: Iterate through each row in the "gamr" DataFrame
    for _, gamr_row in gamr.iterrows():
        gamr_begin = gamr_row["Begin Time - ss.msec"]
        gamr_end = gamr_row["End Time - ss.msec"]
        
        # Step 4: Find overlapping rows in the "oracle" DataFrame
        overlapping_rows = segments[(segments["End"] >= gamr_begin) & (segments["Start"] <= gamr_end)]
        
        if len(overlapping_rows) == 0:
            non_overlapping.append(gamr_row)

Actions

In [8]:
for group in range(1,11):
    print(group)
    actions = pd.read_csv(f"Group_{group:02d}/GROUP_{group:02d}_Actions.csv")
    # print(len(com))
    segments = pd.read_csv(f"Group_{group:02d}/Group_{group:02d}_Oracle.csv")


    mapped_rows = []

    # Step 3: Get the columns to map from the "cps" DataFrame (excluding "Start," "End," and "Utterance")
    columns_to_map = actions.columns.difference(["Begin Time - ss.msec", "End Time - ss.msec", "Duration - ss.msec"])

    # Step 4: Iterate through each row in the "segments" DataFrame
    for _, segment_row in segments.iterrows():
        segment_start = segment_row["Start"]
        segment_end = segment_row["End"]
        
        # Step 5: Find overlapping rows in the "cps" DataFrame
        overlapping_rows = actions[(actions["End Time - ss.msec"] >= segment_start) & (actions["Begin Time - ss.msec"] <= segment_end)]
        
        if not overlapping_rows.empty:
            # Step 6: Initialize a dictionary to store mapped data
            mapped_data = {
                "Utterance": segment_row["Utterance"],
                "Start": segment_start,
                "End": segment_end
            }
            
            # Step 7: Map the values from the "actions" DataFrame to the "segments" row
            for col in columns_to_map:
                mapped_data[col] = overlapping_rows[col].values[0]
            
            # Step 8: Append the mapped data to the list of mapped rows
            mapped_rows.append(mapped_data)
        else:
            # If there are no overlapping rows, create a mapping with empty values
            mapped_data = {
                "Utterance": segment_row["Utterance"],
                "Start": segment_start,
                "End": segment_end
            }
            for col in columns_to_map:
                mapped_data[col] = None
            mapped_rows.append(mapped_data)

    # Step 9: Create a new DataFrame with the mapped rows
    mapped_df = pd.DataFrame(mapped_rows)

    mapped_df.sort_values(by="Utterance").to_csv(f"Group_{group:02d}/GROUP_{group:02d}_Actions_Oracle.csv", index=False)

1
2
3
4
5
6
7
8
9
10
